In [ ]:
 from dotenv import load_dotenv 
    import os 
    import pandas as pd 
    import csv 

 

    # Load environment variables from .env file 
    load_dotenv() 

 

    # Set OpenAI API key 
    openai.api_key = os.getenv("OPENAI_API_KEY") 

 

    # Path to the CSV file with image names 
    csv_file = r"C:\Users\user\Desktop\Thesis\Stage 3 - Data Processing\Sample\sample_memes\main_names.csv" 
    START_INDEX = 0 
    END_INDEX = 100 
    image_names = pd.read_csv(csv_file)["image_name"].tolist()[START_INDEX:END_INDEX] 

 

    # Open a CSV file for appending results 
    with open("sarcasm_classification_results.csv", mode="w", newline="", encoding="utf-8") as csvfile: 
        # Define the CSV writer 
        csv_writer = csv.writer(csvfile) 
         
        # Write the headers 
        csv_writer.writerow(["image_name", "Sarcasm_type", "Explanation"]) 

 

        # Loop through the image names 
        for i, image_name in enumerate(image_names): 
            print(f"Processing image {i + 1} of {len(image_names)}: {image_name}") 

 

            image_url = f"https://raw.githubusercontent.com/sirojiddin4/mathe_sample_test/main/{image_name}" 

 

            prompt = f""" 
            Classify the following meme into one of the sarcasm groups below, considering both visual and textual information: 

 

            not_sarcastic 
            general 
            twisted_meaning 
            very_twisted 

 

            Return: 
            SarcasmGroup: One of the four categories. 
            Explanation: Max 20 words, no commas. 

 

            Output (CSV): 
            SarcasmGroup,Explanation 

 

            Rules: 
            No extra text or comments. 
            Follow the format exactly. 
            Don't include headers of csv 
            Don't include ``` 
                             
            Example: 
            not_sarcastic,A heartfelt message about self-happiness and solitude 
            """  
                    # Make the API call 
        response = openai.ChatCompletion.create( 
            model="gpt-4o", 
            messages=[ 
                { 
                    "role": "user", 
                    "content": [ 
                        {"type": "text", "text": prompt}, 
                        { 
                            "type": "image_url", 
                            "image_url": { 
                                "url": image_url 
                            } 
                        } 
                    ] 
                } 
            ], 
            max_tokens=300, 
        ) 

 

        # Extract the response content 
        result = response.choices[0].message["content"] 
        try: 
            # Split the result into SarcasmGroup and Explanation 
            sarcasm_group, explanation = result.split(",", 1) 
        except Exception as e: 
             # If splitting fails, assign fallback values 
            print(f"Error splitting response for {image_name}: {e}") 
            sarcasm_group = "not identified" 
            explanation = "not identified" 

 

        # Write the data into the CSV 
        csv_writer.writerow([image_name, sarcasm_group.strip(), explanation.strip()]) 
        print(f"Saved result for {image_name}: {sarcasm_group}, {explanation}") 

 import openai 
from dotenv import load_dotenv 
import os 
import pandas as pd 
import csv 

 

# Load environment variables from .env file 
load_dotenv() 

 

# Set OpenAI API key 
openai.api_key = os.getenv("OPENAI_API_KEY") 

 

# Path to the CSV file with textual data 
csv_file = r"C:\Users\user\Desktop\Thesis\Stage 3 - Data Processing\Sample\sample_data_humans.csv" 
START_INDEX = 0 
END_INDEX = 100 
data = pd.read_csv(csv_file) 

 

# Open a CSV file for appending results 
with open("sarcasm_text_classification_results.csv", mode="w", newline="", encoding="utf-8") as csvfile: 
    # Define the CSV writer 
    csv_writer = csv.writer(csvfile) 
     
    # Write the headers 
    csv_writer.writerow(["image_name", "text_corrected", "sarcasm_type", "explanation"]) 

 

    # Loop through the data 
    for i, row in data.iloc[START_INDEX:END_INDEX].iterrows(): 
        image_name = row["image_name"] 
        text_corrected = row["text_corrected"] 
        print(f"Processing entry {i + 1} of {END_INDEX}: {image_name}") 

 

        prompt = f""" 
        Classify the following text into one of the sarcasm groups below: 
         
        Text: "{text_corrected}" 

 

        Sarcasm groups: 
        - not_sarcastic 
        - general 
        - twisted_meaning 
        - very_twisted 

 

        Return: 
        SarcasmType: One of the four categories. 
        Explanation: Max 20 words, no commas. 

 

        Output (CSV): 
        SarcasmType,Explanation 

 

        Rules: 
        No extra text or comments. 
        Follow the format exactly. 
        Don't include headers of csv 
        """  

         # Make the API call 
        try: 
            response = openai.ChatCompletion.create( 
                model="gpt-4o", 
                messages=[ 
                    {"role": "user", "content": prompt} 
                ], 
                max_tokens=300, 
            ) 

 

            # Extract the response content 
            result = response.choices[0].message["content"] 
             
            # Split the result into SarcasmType and Explanation 
            sarcasm_type, explanation = result.split(",", 1) 
        except Exception as e: 
            # If something goes wrong, log the issue and assign fallback values 
            print(f"Error processing {image_name}: {e}") 
            sarcasm_type = "not identified" 
            explanation = "not identified" 

 

        # Write the data into the CSV 
        csv_writer.writerow([image_name, text_corrected, sarcasm_type.strip(), explanation.strip()]) 
        print(f"Saved result for {image_name}: {sarcasm_type}, {explanation}") 

 

 # ============================================ 
#   IMPORT LIBRARIES 
# ============================================ 
import pandas as pd 
import numpy as np 

 

# Scikit-learn for classification metrics 
from sklearn.metrics import ( 
    confusion_matrix, 
    classification_report, 
    cohen_kappa_score 
) 

 

# SciPy and statsmodels for Chi-Squared and McNemar's Test 
from scipy.stats import chi2_contingency 
from statsmodels.stats.contingency_tables import mcnemar 

 

# ============================================ 
#   LOAD DATA 
# ============================================ 
df = pd.read_csv("/kaggle/input/memotion-7k-ai-vs-human-annotations/combined_data.csv") 

 

# Optional: create length columns (word counts) for text & explanations 
df["text_length"] = df["text_corrected"].astype(str).apply(lambda x: len(x.split())) 
df["multimodal_explanation_length"] = df["multimodal_explanation_ai"].astype(str).apply(lambda x: len(x.split())) 
df["unimodal_explanation_length"] = df["unimodal_explanation_ai"].astype(str).apply(lambda x: len(x.split())) 

 

# NOTE: Updated the label_set to match alphabetical order of classes 
#       ("general" likely comes before "not_sarcastic"). 
#       Verify with: sorted(df["multimodal_annotation_humans"].unique()) 
label_set = ["general", "not_sarcastic", "twisted_meaning", "very_twisted"] 

 

# ============================================ 
#   HELPER FUNCTIONS 
# ============================================ 
def calculate_overlap(ai_labels, human_labels): 
    """ 
    Calculate simple percentage overlap between AI and human labels. 
    """ 
    matches = (ai_labels == human_labels) 
    overlap_percent = 100.0 * matches.sum() / len(matches) 
    return overlap_percent 

 

def print_confusion_metrics(ai_labels, human_labels, label_set=None, desc=""): 
    """ 
    Print confusion matrix, classification report, and Cohen's Kappa for AI vs. Human labels. 
    """ 
    print(f"\n=== {desc} ===") 

     if label_set is not None: 
        cm = confusion_matrix(human_labels, ai_labels, labels=label_set) 
    else: 
        cm = confusion_matrix(human_labels, ai_labels) 

 

    cm_df = pd.DataFrame( 
        cm, 
        index=[f"True: {label}" for label in label_set], 
        columns=[f"Predicted: {label}" for label in label_set] 
    ) 

 

    print("Confusion Matrix:") 
    print(cm_df) 

 

    print("\nClassification Report:") 
    if label_set is not None: 
        print(classification_report(human_labels, ai_labels, target_names=label_set, zero_division=0)) 
    else: 
        print(classification_report(human_labels, ai_labels, zero_division=0)) 

 

    kappa = cohen_kappa_score(human_labels, ai_labels) 
    print(f"Cohen's Kappa: {kappa:.3f}") 

 

def run_chi_squared_test(cat_var, numeric_or_cat_var, desc=""): 
    """ 
    Perform Chi-Squared test between a categorical variable (cat_var) and 
    another variable (numeric_or_cat_var), which you can treat as categorical 
    (e.g. binned or direct word counts). 
    """ 
    print(f"\n--- Chi-Squared Test: {desc} ---") 

 

    # Create a contingency table 
    contingency_table = pd.crosstab(cat_var, numeric_or_cat_var) 
    print("Contingency Table (first few rows/cols):") 
    print(contingency_table.iloc[:10, :10])  # truncated if large 

 

    chi2, p, dof, expected = chi2_contingency(contingency_table) 

 

    print(f"\nChi-Squared Statistic = {chi2:.3f}") 
    print(f"Degrees of Freedom = {dof}") 
    print(f"p-value = {p:.6f}") 

 

    if p < 0.05: 
        print("=> Reject null hypothesis: There is a significant association.") 
    else: 
        print("=> Fail to reject null hypothesis: No significant association.") 

 

def run_mcnemar_test(unimodal_correct, multimodal_correct): 
    """ 
    Perform McNemar's test on two paired nominal outcomes: unimodal_correct vs. multimodal_correct. 
    """ 
    a = int((unimodal_correct & multimodal_correct).sum()) 
    b = int((unimodal_correct & ~multimodal_correct).sum()) 
    c = int((~unimodal_correct & multimodal_correct).sum()) 
    d = int((~unimodal_correct & ~multimodal_correct).sum()) 

   table = [[a, b], 
             [c, d]] 

 

    result = mcnemar(table, exact=False, correction=True) 

 

    print("\n--- McNemar's Test ---") 
    print("Contingency Table (Correct vs. Incorrect):") 
    print(f"                 Multimodal Correct   Multimodal Wrong") 
    print(f"Unimodal Correct          {a}                  {b}") 
    print(f"Unimodal Wrong            {c}                  {d}") 

 

    print(f"\nStatistic = {result.statistic:.3f}, p-value = {result.pvalue:.6f}") 
    if result.pvalue < 0.05: 
        print("=> Significant difference between unimodal and multimodal performance.") 
    else: 
        print("=> No significant difference between unimodal and multimodal performance.") 

 

# ============================================ 
#   MULTIMODAL SECTION 
# ============================================ 
print("============================================") 
print("      MULTIMODAL ANALYSIS") 
print("============================================") 

 

# 1) Sample Dataset 
print("\n1) SAMPLE DATASET (Reason: to check correctness and integrity)") 
display(df.head(5)) 

 

# 2) Overlap with Human Annotations 
print("\n2) OVERLAP WITH HUMAN ANNOTATIONS (Reason: to get a brief overview)") 
multi_overlap = calculate_overlap(df["multimodal_annotation_ai"], df["multimodal_annotation_humans"]) 
print(f"Multimodal AI vs Human Overlap: {multi_overlap:.2f}%") 

 

# 3) Cohen’s Kappa 
print("\n3) COHEN’S KAPPA (Reason: to get the actual agreement rate considering chance)") 
kappa_multi = cohen_kappa_score(df["multimodal_annotation_humans"], df["multimodal_annotation_ai"]) 
print(f"Cohen's Kappa (Multimodal vs Human): {kappa_multi:.3f}") 

 

# 4) Classification Report 
print("\n4) CLASSIFICATION REPORT (Reason: to understand performance for each category)") 
print( 
    classification_report( 
        df["multimodal_annotation_humans"], 
        df["multimodal_annotation_ai"], 
        target_names=label_set, 
        zero_division=0 
    ) 
) 

 

# 5) Confusion Matrix 
print("\n5) CONFUSION MATRIX (Reason: to understand categorization patterns and biases)") 
print_confusion_metrics( 
    ai_labels=df["multimodal_annotation_ai"], 
    human_labels=df["multimodal_annotation_humans"], 
    label_set=label_set, 
    desc="Multimodal AI vs. Human" ) # 6) Chi-Squared: Multimodal Annotation VS text_corrected length 
print("\n6) CHI-SQUARED TEST: MULTIMODAL ANNOTATION VS TEXT LENGTH (Reason: to see if input size affects 
outcome)") 
run_chi_squared_test( 
    cat_var=df["multimodal_annotation_ai"], 
    numeric_or_cat_var=df["text_length"], 
    desc="Multimodal Annotation vs. Text Word Count" 
) 

 

# 7) Chi-Squared: Multimodal Annotation VS multimodal_explanation_ai length 
print("\n7) CHI-SQUARED TEST: MULTIMODAL ANNOTATION VS EXPLANATION LENGTH (Reason: to see if label affects 
explanation)") 
run_chi_squared_test( 
    cat_var=df["multimodal_annotation_ai"], 
    numeric_or_cat_var=df["multimodal_explanation_length"], 
    desc="Multimodal Annotation vs. Multimodal Explanation Length" 
) 

 

# ============================================ 
#   UNIMODAL SECTION 
# ============================================ 
print("\n\n============================================") 
print("      UNIMODAL ANALYSIS") 
print("============================================") 

 

# 1) Sample Dataset 
print("\n1) SAMPLE DATASET (Reason: to check correctness and integrity)") 
display(df.head(5)) 

 

# 2) Overlap with Human Annotations 
print("\n2) OVERLAP WITH HUMAN ANNOTATIONS (Reason: to get a brief overview)") 
uni_overlap = calculate_overlap(df["unimodal_annotation_ai"], df["multimodal_annotation_humans"]) 
print(f"Unimodal AI vs Human Overlap: {uni_overlap:.2f}%") 

 

# 3) Cohen’s Kappa 
print("\n3) COHEN’S KAPPA (Reason: to get the actual agreement rate considering chance)") 
kappa_uni = cohen_kappa_score(df["multimodal_annotation_humans"], df["unimodal_annotation_ai"]) 
print(f"Cohen's Kappa (Unimodal vs Human): {kappa_uni:.3f}") 

 

# 4) Classification Report 
print("\n4) CLASSIFICATION REPORT (Reason: to understand performance for each category)") 
print( 
    classification_report( 
        df["multimodal_annotation_humans"], 
        df["unimodal_annotation_ai"], 
        target_names=label_set, 
        zero_division=0 
    ) 
) 

 

# 5) Confusion Matrix 
print("\n5) CONFUSION MATRIX (Reason: to understand categorization patterns and biases)") 
print_confusion_metrics( 
    ai_labels=df["unimodal_annotation_ai"], 
    human_labels=df["multimodal_annotation_humans"], 
    label_set=label_set, 
        desc="Unimodal AI vs. Human" 
) 

 

# 6) Chi-Squared: Unimodal Annotation vs text_corrected length 
print("\n6) CHI-SQUARED TEST: UNIMODAL ANNOTATION VS TEXT LENGTH (Reason: to see if input size affects outcome)") 
run_chi_squared_test( 
    cat_var=df["unimodal_annotation_ai"], 
    numeric_or_cat_var=df["text_length"], 
    desc="Unimodal Annotation vs. Text Word Count" 
) 

 

# 7) Chi-Squared: Unimodal Annotation vs unimodal_explanation_ai length 
print("\n7) CHI-SQUARED TEST: UNIMODAL ANNOTATION VS EXPLANATION LENGTH (Reason: to see if label affects 
explanation)") 
run_chi_squared_test( 
    cat_var=df["unimodal_annotation_ai"], 
    numeric_or_cat_var=df["unimodal_explanation_length"], 
    desc="Unimodal Annotation vs. Unimodal Explanation Length" 
) 

 

# ============================================ 
#   SPECIAL ANALYSIS: MCNEMAR'S TEST 
# ============================================ 
print("\n\n============================================") 
print("      SPECIAL: MCNEMAR’S TEST") 
print("============================================") 
print("(Reason: to check if there is a significant difference between unimodal and multimodal annotations on the SAME 
items)") 

 

unimodal_correct = (df["unimodal_annotation_ai"] == df["multimodal_annotation_humans"]) 
multimodal_correct = (df["multimodal_annotation_ai"] == df["multimodal_annotation_humans"]) 

 

run_mcnemar_test(unimodal_correct, multimodal_correct) 

 

print("\nAll calculations are complete.") 
                  # ============================================ 
#   IMPORT LIBRARIES FOR HEATMAPS 
# ============================================ 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
import pandas as pd 

 

df = pd.read_csv("/kaggle/input/memotion-7k-ai-vs-human-annotations/combined_data.csv") 

 

# ============================================ 
#   FUNCTION TO GENERATE AND SAVE HEATMAPS 
# ============================================ 
def save_confusion_matrix_heatmap(cm, labels, title, filename): 
    """ 
    Generate and save a heatmap for a confusion matrix. 
    """ 
    plt.figure(figsize=(8, 6)) 
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels) 
    plt.title(title) 
    plt.xlabel("Predicted") 
    plt.ylabel("True") 
    plt.savefig(filename, bbox_inches="tight", dpi=300) 
    plt.close() 

 

# ============================================ 
#   GENERATE AND SAVE HEATMAPS 
# ============================================ 
# Multimodal Confusion Matrix Heatmap 
cm_multi = confusion_matrix(df["multimodal_annotation_humans"], df["multimodal_annotation_ai"], labels=label_set) 
save_confusion_matrix_heatmap( 
    cm_multi, 
    label_set, 
    title="Multimodal AI vs Human Confusion Matrix", 
    filename="multimodal_confusion_matrix.png" 
) 

 

# Unimodal Confusion Matrix Heatmap 
cm_uni = confusion_matrix(df["multimodal_annotation_humans"], df["unimodal_annotation_ai"], labels=label_set) 
save_confusion_matrix_heatmap( 
    cm_uni, 
    label_set, 
    title="Unimodal AI vs Human Confusion Matrix", 
    filename="unimodal_confusion_matrix.png" 
) 

 

print("\nHeatmaps for confusion matrices have been saved as 'multimodal_confusion_matrix.png' and 
'unimodal_confusion_matrix.png'.") 

 # ============================================ 
#   HISTOGRAMS & BASIC CORRELATION CHECKS 
# ============================================ 

 

df["text_length"] = df["text_corrected"].astype(str).apply(lambda x: len(x.split())) 
df["multimodal_explanation_length"] = df["multimodal_explanation_ai"].astype(str).apply(lambda x: len(x.split())) 
df["unimodal_explanation_length"] = df["unimodal_explanation_ai"].astype(str).apply(lambda x: len(x.split())) 

 

 

# ============================================ 
#   PRINT AVERAGE LENGTHS PER LABEL 
# ============================================ 

 

# A) Average Text Length per Label 
print("=== Average Text Length by Multimodal Classification ===") 
print(df.groupby("multimodal_annotation_ai")["text_length"].mean()) 
print("\n=== Average Text Length by Unimodal Classification ===") 
print(df.groupby("unimodal_annotation_ai")["text_length"].mean()) 

 

# B) Average Explanation Length per Label 
print("\n=== Average Multimodal Explanation Length by Multimodal Classification ===") 
print(df.groupby("multimodal_annotation_ai")["multimodal_explanation_length"].mean()) 
print("\n=== Average Unimodal Explanation Length by Unimodal Classification ===") 
print(df.groupby("unimodal_annotation_ai")["unimodal_explanation_length"].mean()) 

 